# 细节

## 交叉熵

在分类任务中我们不能单纯的将mse输入回去. 因为很可能输入回去的错误会使得梯度消失. 比如: 我们有3个正确2个错误, 计算回去的mse是一个值, 然后经过计算以后, 我们的权重w更新了, 但是输出结果还是3个正确2个错误, 因此mse并没有改变. 此时不断地修正w很可能导致梯度消失.

- sigmoid + mse 很可能导致梯度小时
- 收敛很慢

> 需要注意的是并不是一定不要用mse, mse在很多种情况下还是非常的优秀的. 只是他会造成很多问题, 但是它计算起来会很简单, 比如metalearning中使用mse会计算很快.

因此为了避免这种情况我们需要使用交叉熵


In [7]:
import torch
import torch.nn.functional as F


In [8]:
x = torch.randn(1, 784)
w = torch.randn(10, 784)

logits = x@w.t()  # 自己计算一次预测
print(logits.shape)

# 使用softmax计算一次结果
pred = F.softmax(logits, dim=1)
print(pred.shape)

# 使用交叉熵必须使用logits, 不能使用pred因为交叉熵已经继承了softmax
print(F.cross_entropy(logits, torch.tensor([3])))

# 如果一定要使用交叉熵, 可以使用none negative进行预测. 但是必须进行一次log算法
# 得到一个log的predict
pred_log = torch.log(pred)
print(F.nll_loss(pred_log, torch.tensor([3])))


torch.Size([1, 10])
torch.Size([1, 10])
tensor(76.0477)
tensor(76.0477)


## 多分类问题

使用交叉熵优化多分类问题

In [9]:
import torch.optim as optim
from torch import nn
from torchvision import datasets, transforms

# 输入层
w1, b1 = torch.randn(200, 784, requires_grad=True), torch.zeros(
    200, requires_grad=True)
w2, b2 = torch.randn(200, 200, requires_grad=True), torch.zeros(
    200, requires_grad=True)
# 输出层
w3, b3 = torch.randn(10, 200, requires_grad=True), torch.zeros(
    10, requires_grad=True)

# 添加kaiming初始化
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)


# 使用relu函数进行前向传播辅助计算
def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    x = F.relu(x)
    return x


In [10]:
# 定义优化器
learning_rate = 0.01
optimizer = optim.SGD([w1, b1, w2, b2, w3, b3], lr=learning_rate)
criteon = nn.CrossEntropyLoss()


In [11]:
# 定一mnist数据读取器
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)


In [12]:
epochs = 10
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)

        # forward中不能出现softmax因为后面我们要用entropy来计算, 其中已经包含了softmax了
        logits = forward(data)
        loss = criteon(logits, target)  # 使用交叉熵计算loss, 而不是mse

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.569833
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.055235
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.910035

Test set: Average loss: 0.0042, Accuracy: 7083/10000 (71%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.801024
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.773433
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.749979

Test set: Average loss: 0.0038, Accuracy: 7235/10000 (72%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.749595
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.635862
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.771702

Test set: Average loss: 0.0036, Accuracy: 7329/10000 (73%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.865054
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.687739
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.712944

Test set: Average loss: 0.0035, Accuracy: 7386/10000 (74%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.788834
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.668117
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.641329

Test set: Average loss: 0.0024

我们如果不使用数据初始化使用的就是高斯初始化, 此时可以发现上面的lose和准确率被锁定在10%. loss一直不变. 

这里, 我们的网络结构非常的简单, 同时我们使用的是relu函数, relu函数不会出现梯度离散的情况. **loss信息长时间不变表明了梯度信息接近于0**, 因此我们需要明白为什么梯度为0.

出现梯度为0的原因一方面是learning rate过大导致gradient vanish, 另一方面就是数据初始化问题.

此时我们使用kaiming的数据初始化.

In [13]:
# 使用kaiming初始化, 初始化所有的权重
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)


tensor([[-0.0257,  0.0396, -0.0005,  ...,  0.0971, -0.0170,  0.0907],
        [-0.1090, -0.1282, -0.0603,  ...,  0.0797,  0.0831,  0.0948],
        [-0.0545, -0.0405,  0.0252,  ...,  0.0263, -0.0316, -0.2278],
        ...,
        [ 0.0206, -0.0957, -0.0516,  ...,  0.0515, -0.0398,  0.0632],
        [-0.0486, -0.0016,  0.1319,  ..., -0.0299,  0.1092,  0.0035],
        [ 0.1982,  0.0827, -0.0673,  ...,  0.1431, -0.0279, -0.0217]],
       requires_grad=True)

## 激活函数以及gpu加速

除了relu函数意外, 我们可以使用leakyrelu函数. Relu函数不容易出现梯度离散的情况因为输出的结果相对于输入是相通的, 但是也只是"不容易"而不是不会. 因此,相比于relu函数, leakyrelu就会更不容易出现离散的情况.

```python
self.model = nn.Sequential(
    nn.Linear(784, 200),
    nn.LeakyReLU(inplace=True),
    nn.Linear(200, 200),
    nn.LeakyReLU(inplace=True),
    nn.Linear(200, 10),
    nn.LeakyReLU(inplace=True),
)
```

relu函数其实还是不连续的,因此我们可以使用另一个函数`SELU`, 他和RELU函数相似但是是具有数学意义的, 更加光滑(很少用)

![selu](./assets/18.png)

softplus也是relu的加强版, 是完全的数学函数(很少用)

![0](./assets/19.png)

### gpu加速

`.to(device)`方法返回一个reference.

```python
# 如果使用mac的话使用mps gpu芯片
# device = torch.device("mps") 
device = torch.device('cuda:0')
# 对模块使用.to(device)方法调用和之前的使用方式是一摸一样的, 是一个inplace的操作
net = MLP().to(device) 
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device) # loss放入gpu

for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        # 如果使用to(device)方法返回的就不是一样的模块
        # 这个数据是可以back反向传播的,但是这个操作会产生两个tensor, 一个是cpu上, 一个是gpu上
        data, target = data.to(device), target.cuda() # 两种方法, 一般使用第一种方法
```

## 测试

Generalize Performance 过拟合

需要引用测试集.

- test on epoch
- test on step


In [15]:
logits = torch.rand(4, 10)
pred = F.softmax(logits, dim=1)
print(pred.shape)
pred_label = pred.argmax(dim=1)
print(pred_label)

print(logits.argmax(dim=1))

label = torch.tensor([9, 3, 2, 4])
correct = torch.eq(pred_label, label) # 计算相等的元素
print(correct)
print(correct.sum().float().item()/4)


torch.Size([4, 10])
tensor([5, 3, 2, 2])
tensor([5, 3, 2, 2])
tensor([False,  True,  True, False])
0.5


一般在epoch中直接进行test

```python
net.eval()
test_loss = 0
correct = 0
# 每一个test文件输入到网络文件中
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.argmax(dim=1)
    correct += pred.eq(target).float().sum().item()

# 正确数据除以总数据集
test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
```

## 可视化

Visdom可视化. Tensorflow 使用tensorboard来进行展示, pytorch使用tensorboardx来进行展示.

```shell
pip install tensorboardx
```

Tensorboard 的本质就是一个服务器, 然后我们将数据写入那个服务器. 但是有一个问题就是writer是作用在cpu上的, 我们需要将数据转换到cpu上, 然后才能通过cpu写入数据. 而且Tensorboard会将数据写入硬盘, 占用资源会非常大. 而且每30秒更新一次.

```python
from tensorboardX import SummaryWriter
writer = SummaryWriter()

writer.add_scaler('data/scalar1', dummy_s1[0], n_iter)

writer.add_scalers(
    'data/scalar_group',
    {'xsinx': n_iter*np.sin(n_iter),
     "xcosx": n_iter*np.cos(n_iter),
     "arctanx": np.arctan(n_iter)}, n_iter)

writer.add_image('Image',x,n_iter)
writer.add_text('Text','text logged at step' + str(n_iter), n_iter)

writer.close()
```



Visdom的优势在于可以接收原生的tensor. 

```shell
pip install visdom
python -m visdom.server # 开启服务器进程
```

如果不行, 就去下载服务器, 或者直接下载源代码进行编译. 如果下载源代码就进入源代码目录使用`pip install -e .`来安装. 

也可以考虑docker安装. 

`docker run -it -p 8097:8097 --name visdom hypnosapos/visdom -d`

注意docker直接下载也可能出现芯片错误, 可以自己编译

```python
from visdom import Visdom
viz = Visdom()
# 第一个参数是y, 第二个参数是x, win哪一个窗口, 如果不指定就是用默认窗口main
viz = line([0.],[0.], win="train_loss", opts=dict(title="train lose"))
# 然后一次一次添加数据, update=append表示当前这个是追加动作
# 同样, 第一个值是y,第二个值是x, win哪一个窗口
# 注意这里也是一个numpy数据
viz.line([loss.item()],[global_step],win='train_loss', update='append')
```

多条线进行数据输入

```python
from visdom import Visdom
viz = Visdom()
viz.line([[test_loss, correct / len(test_loader.dataset)]],
            [global_step], win='test', update='append')
viz.images(data.view(-1, 1, 28, 28), win='x')
viz.text(str(pred.detach().cpu().numpy()), win='pred',
            opts=dict(title='pred'))
```